In [1]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import pandas as pd 
pd.options.display.max_rows = 8  
import requests
import parsel
from bs4 import BeautifulSoup
from lxml import etree
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome,ChromeOptions
from selenium.webdriver.common.by import By
import warnings
import csv
import random
import os

#忽略警告
warnings.filterwarnings("ignore")

1

In [2]:
# 导入所需的库
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import os
import json

# 设置请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 定义要提取的年份范围
years = range(2013, 2024)

# 初始化链接字典
link_dict = {}

# 确保实验一爬取数据文件夹存在
os.makedirs("./实验一爬取数据", exist_ok=True)

# 循环处理每个年份
for year in years:
    # 构造年份对应的索引页面URL
    index_url = f"https://www.cnipa.gov.cn/tjxx/jianbao/year{year}/indexy.html"
    
    try:
        # 发送HTTP请求获取索引页面内容
        response = requests.get(index_url, headers=headers)
        response.raise_for_status()  # 检查请求是否成功
    except requests.exceptions.RequestException as e:
        print(f"获取索引页面 {index_url} 失败: {e}")
        continue

    # 解析索引页面，寻找包含特定关键字的链接
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)
    
    # 筛选包含特定关键字的链接
    target_keywords = ['专利申请代理状况', '专利申请、授权按IPC分类分布状况', '商标申请、注册及有效注册状况']
    target_link_pages = []
    for link in links:
        # 检查链接文本是否包含目标关键词之一
        if any(keyword in link.text for keyword in target_keywords):
            href = link['href']
            # 构造完整的链接URL
            full_url = urljoin(index_url, href)
            target_link_pages.append(full_url)
    
    # 对每个目标链接页面进行进一步处理
    link_dict[year] = []  # 初始化当年份的列表
    for target_link_page in target_link_pages:
        try:
            # 发送HTTP请求获取目标链接页面内容
            target_response = requests.get(target_link_page, headers=headers)
            target_response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"获取目标链接页面 {target_link_page} 失败: {e}")
            continue
        
        # 解析目标链接页面，提取其中的所有链接
        target_soup = BeautifulSoup(target_response.text, 'html.parser')
        sub_links = target_soup.find_all('a', href=True)
        
        # 过滤并整理子链接，排除包含 'indexy' 的链接
        sub_target_links = []
        for sub_link in sub_links:
            sub_href = sub_link['href']
            # 过滤掉非HTML文件的链接以及包含 'indexy' 的链接
            if sub_href.endswith('.html') and 'indexy' not in sub_href:
                # 构造完整的子链接URL
                sub_full_url = urljoin(target_link_page, sub_href)
                sub_target_links.append(sub_full_url)
        
        # 假设目标链接页面的标题作为大类名称
        # 尝试从目标链接页面中提取大类名称
        class_name = target_soup.find('title')
        if class_name:
            class_name = class_name.text.strip()
        else:
            # 如果没有标题，使用链接中的文件名作为大类名称
            class_name = os.path.basename(target_link_page).split('.')[0]
        
        # 将大类名称和子链接数组存储到链接字典中
        link_dict[year].append({
            'class_name': class_name,
            'links': sub_target_links
        })
    
    print(f"已处理年份 {year} 的链接")

# 保存链接字典到JSON文件
with open('./实验一爬取数据/link_dict.json', 'w', encoding='utf-8') as f:
    json.dump(link_dict, f, ensure_ascii=False, indent=4)

print("链接字典已保存到 ./实验一爬取数据/link_dict.json")

已处理年份 2013 的链接
已处理年份 2014 的链接
已处理年份 2015 的链接
已处理年份 2016 的链接
已处理年份 2017 的链接
已处理年份 2018 的链接
已处理年份 2019 的链接
已处理年份 2020 的链接
已处理年份 2021 的链接
已处理年份 2022 的链接
已处理年份 2023 的链接
链接字典已保存到 ./实验一爬取数据/link_dict.json


In [3]:
import json
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 设置请求头，模拟浏览器访问
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}

failed_links = []

def crawl_and_process(year, class_name, link, table_title):
    """
    爬取并处理网页表格数据，并保存为CSV文件
    """
    # 创建对应年份的文件夹（如果不存在的话）
    folder_path = f'实验一爬取数据/{year}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 发送 GET 请求获取网页内容
    response = requests.get(link, headers=headers)
    # 设置正确的编码格式
    response.encoding = 'utf-8'
    # 使用 BeautifulSoup 解析 HTML 内容
    soup = BeautifulSoup(response.text, 'html.parser')
    # 查找所有表格
    tables = soup.find_all('table')
    
    # 检查是否存在表格
    if not tables:
        print(f"No tables found in {link}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "No tables found"})
        return
    
    # 尝试获取第二个表格（根据之前的代码逻辑）
    specified_tables = tables[1] if len(tables) > 1 else tables[0]
    
    # 将表格转换为 DataFrame
    try:
        df = pd.read_html(str(specified_tables))[0]
    except Exception as e:
        print(f"Failed to parse table in {link}: {e}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": str(e)})
        return
    
    # 对表格进行数据处理
    if len(df) > 4:
        df = df[4:]
        # 检查索引是否存在，避免越界错误
        indices_to_drop = [idx for idx in [4, 6] if idx in df.index]
        df.drop(indices_to_drop, inplace=True)
    
    # 检查 DataFrame 是否为空
    if df.empty:
        print(f'No data to save for {class_name}-{table_title}')
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "Empty DataFrame"})
        return
    
    # 检查列数是否一致
    if len(df.columns) < 1:
        print(f'No columns found in DataFrame for {class_name}-{table_title}')
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "No columns found"})
        return
    
    # 尝试进行更复杂的数据处理
    try:
        # 设置第一列作为索引
        df.set_index(df.columns[0], inplace=True)
        df.index.name = None
        
        # 将第一行作为列名
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        df.columns.name = None
        
        # 将原有索引设置为一级索引，同时将第一列和第一行作为二级索引
        df = df.set_index(df.columns[0], append=True)
        df.index.name = None  # 删除索引列的名称
        
        # 创建二级索引
        df.columns = pd.MultiIndex.from_arrays([df.iloc[0], df.columns], names=[None, None])
        df = df.iloc[1:]  # 删除原始的第一行
        df = df.swaplevel(axis=1)
        
        # 保存为 CSV 文件，如果 DataFrame 不为空
        if not df.empty:
            df.to_csv(f'{folder_path}/{class_name}-{table_title}.csv', encoding='utf-8-sig')
            print(f'Successfully saved {class_name}-{table_title}.csv')
        else:
            print(f'No data to save for {class_name}-{table_title}')
            failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "Empty DataFrame after processing"})
    except Exception as e:
        print(f"Failed to process {class_name}-{table_title}: {e}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": str(e)})

# 加载 JSON 文件
with open(r'实验一爬取数据/link_dict.json', 'r', encoding='utf-8') as f:
    link_dict = json.load(f)

# 遍历每个年份和类别链接，进行爬取和处理
for year in link_dict:
    if link_dict[year]:  # 如果该年份的类别列表不为空
        for class_info in link_dict[year]:
            class_name = class_info["class_name"]
            for link in class_info["links"]:
                # 从链接中提取表格标题
                table_title = link.split('/')[-1].replace('.html', '')
                crawl_and_process(year, class_name, link, table_title)

# 保存处理失败的链接到 JSON 文件
with open(r'实验一爬取数据/failed_links.json', 'w', encoding='utf-8') as f:
    json.dump(failed_links, f, ensure_ascii=False, indent=4)

print(f"处理失败的链接已保存到 '实验一爬取数据/failed_links.json'")

Failed to process 专利申请代理状况-d1: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d2: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d3: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d4: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e1: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e2: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e3: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e4: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e5: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e6: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e7: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e8: index 0 is out of bounds for

2

In [ ]:
import pandas as pd
import requests
import random
from time import sleep

url = 'https://www.baiten.cn/results/filter'

# Select Zhejiang province
province = '浙江'

df = pd.DataFrame(dtype=str)

for year in range(2000, 2025):
    params = {
        'q': 'aa:((' + province + ')) AND (pd:[' + str(year) + '0101 TO ' + str(year) + '1231])',
        'fq': '',
        'sc': '35184372088831'
    }
    
    headers = {
        'Cookie': '_uab_collina=171636084297048745836638; yunsuo_session_verify=651468e84a027b9680fb898cb3261dac; JSESSIONID=467E8A8E43574AD6ADDD2CA1AA8E598B; zlcp2024423=true; BSESSION=e2148644ecde040ae13f44b2ace4184a38f0d6b0623961ad; PD=a6e2323d0e6adfd16b0bb9a0e28457c5d0c7bf66bf1f72327d2d56e14cb82af5a5f1e3e7ae91cf98c7fa13fa30f3d1fd'
    }
    
    try:
        resp = requests.post(url=url, params=params, headers=headers)
        page_json = resp.json()
        
        # Extract patent counts by type
        cn_um = int(page_json['facetPivots']['country']['cn'].get('cn_um', 0))  # Utility model
        cn_id = int(page_json['facetPivots']['country']['cn'].get('cn_id', 0))  # Design
        cn_in = int(page_json['facetPivots']['country']['cn'].get('cn_in', 0))  # Invention
        cn_gp = int(page_json['facetPivots']['country']['cn'].get('cn_gp', 0))  # Granted invention
        cn_in_gp = int(page_json['facetPivots']['country']['cn'].get('cn_in_gp', 0))  # Published invention
        
        # Add to dataframe
        df = pd.concat([df, pd.DataFrame({
            '发明专利': cn_in,
            '发明公开专利': cn_in_gp,
            '发明授权专利': cn_gp,
            '外观专利': cn_id,
            '实用新型专利': cn_um,
            '年份': year,
            '省份': province
        }, index=[len(df)])], axis=0)
        
        print(f'Successfully collected data for {year}')
        sleep(random.randint(1,3))
        
    except Exception as e:
        print(f'Failed to collect data for {year}: {e}')
        continue


# Save to Excel
df.to_excel(f'./实验一爬取数据/佰腾网/佰腾网{province}2000-2024专利数据.xlsx', index=False)
print('Data collection completed')

In [6]:
#####  登录微博
# 导入需要的库
from selenium import webdriver  # Selenium用于模拟浏览器操作
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains  
import time  

In [ ]:
# # 启动浏览器
# browser = webdriver.Chrome()  # 启动Chrome浏览器，确保已经安装Chrome浏览器和对应的ChromeDriver驱动程序

# # 打开微博首页
# browser.get("https://passport.weibo.com/sso/signin?entry=miniblog&source=miniblog&disp=popup&url=https%3A%2F%2Fweibo.com%2Fnewlogin%3Ftabtype%3Dweibo%26gid%3D102803%26openLoginLayer%3D0%26url%3Dhttps%253A%252F%252Fweibo.com%252F")  # 使用浏览器打开微博首页

# # 等待页面加载完成
# time.sleep(3)  

# # 定位要点击的元素
# button = browser.find_element(By.XPATH, "//*[@id='app']/div/div/div[2]/div[2]/ul/li[2]/a/span[2]")  # 用XPath定位元素，可以根据实际情况修改 //*[@id="app"]/div/div/div[2]/div/ul/li[2]/a/span[1]
# # 执行点击操作
# button.click()

# # 定位账号输入框，并填入账号
# user_input = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/form/div[1]/input")  # 使用id属性定位账号输入框
# user_input.send_keys("17763769712")  # 向账号输入框输入你的用户名

# # 定位密码输入框，并填入密码
# pwd_input = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/form/div[2]/input")  # 使用name属性定位密码输入框
# pwd_input.send_keys("Yyzz20030618")  # 向密码输入框输入你的密码

# # 等待登录完成
# time.sleep(3)  # 等待10秒钟，确保登录成功和页面跳转完成，这里的时间可以根据实际情况进行调整

# # 定位要点击的是登录按钮
# button = browser.find_element(By.XPATH,"//*[@id='app']/div/div/div[2]/div[2]/button")
# # 点击登录按钮
# button.click()

# # # 滚动条拉到指定位置（具体元素）  
# # target = browser.find_element(By.XPATH,"//*[@class='slider']")  
# # browser.execute_script("arguments[0].scrollIntoView();", target)  

# time.sleep(3)  
# cookies = browser.get_cookies()
# print(cookies)
# # browser.close() 

NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
######北京知识产权官方微博数据抓取
import requests
import csv
from lxml import etree
import time
import random
from datetime import datetime
import os
import pandas as pd

In [ ]:
def extract_full_content(page):
    headers = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51',
        'cookie': '_T_WM=29007828263; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFPrQQDSM8GU-LDwXGJfO-75JpX5K-hUgL.FoMRSh-f1hqNSoq2dJLoI7p0dNHjdg8X9HvodGnt; SSOLoginState=1687401343; ALF=1689993343; SCF=AliNZPJfd8jrC1kjeXX5Wln4GfwuiZk5G2brx6c1VvQ_NS49PAH7r8V1CrDXL_YlLEmhT_4y5HWR4ywbGEllBaM.; SUB=_2A25Jl8MvDeRhGeFG71cU-CjLzTqIHXVre-1nrDV6PUJbktANLXT_kW1NeWK98E_aztGw8jkuazsPlMRFEQEXXDZy'
    }
    content = page.xpath('./div/span[@class="ctt"]//text()')

    full_content = ''
    for i in range(len(content)):
        full_content += content[i]

    if u'全文' in full_content:
        full_content_link = page.xpath('./div/span[@class="ctt"]/a/@href')
        content_url = 'https://weibo.cn' + full_content_link[-1]
        response = requests.get(url=content_url, headers=headers)
        tree = etree.HTML(response.text.encode('utf-8'))
        div = tree.xpath('//*[@id="M_"]/div[1]/span[@class="ctt"]//text()')
        full_content = ''
        for j in range(len(div)):
            full_content += div[j]
        return full_content
    else:
        return full_content


def format_time(time_info):
    now = datetime.now()
    if '今天' in time_info:
        time_str = time_info.replace('今天', now.strftime('%Y-%m-%d'))
    elif '月' in time_info and '日' in time_info:
        month = time_info.split('月')[0]
        day = time_info.split('月')[1].split('日')[0]
        time_str = now.strftime('%Y') + '-' + month.zfill(2) + '-' + day.zfill(2)
    else:
        time_str = time_info
    return time_str


def extract_info(page):
    content = extract_full_content(page)
    time_info = page.xpath('.//span[@class="ct"]/text()')[0].split('\xa0')[0]
    time_str = format_time(time_info)
    source = page.xpath('.//span[@class="ct"]/text()')[0].split('\xa0')[1][2:]
    like_count = page.xpath('.//a[contains(text(), "赞[")]/text()')[0][2:-1]
    comment_count = page.xpath('.//a[contains(text(), "评论[")]/text()')[0][3:-1]
    repost_count = page.xpath('.//a[contains(text(), "转发[")]/text()')[0][3:-1]
    return content, time_str, source, like_count, comment_count, repost_count

In [ ]:
url = 'https://weibo.cn/2611704875?mp=1302&page='
base_url = 'https://weibo.cn/2611704875?mp=1302&page='

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0',
    'cookie': '_T_WM=8f1c0a26d8ada0f4491205a111d59473; SCF=Alzst0ONGEqytA_Ri0CzQj0KKN9QRFV9lcOiz2ZJ7xKiYrZnDRpnPqWdtgbyG8KAn0wfqZ9VtQPZk8EAnEHRy7I.; SUB=_2A25LSxo6DeRhGeFH6lsZ9yvIyD-IHXVoKRPyrDV6PUJbktANLWjwkW1Ne9dAHCZXYumFfl_qjyxmUhO7cAm8qCoA; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWJLVu-LI4uxDCyKD4XGJqV5JpX5KMhUgL.FoM4eK.RS0-Xe0e2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMfeo2NSh.ESh-X; SSOLoginState=1716480618; ALF=1719072618'
}

filename = '北京知识产权微博内容.csv'

# Delete the existing file if it exists
if os.path.exists(filename):
    os.remove(filename)
    print('Existing data.csv file deleted.')

with open(filename, mode='a', encoding='utf-8', newline='') as file:
    csv_writer = csv.writer(file)
    page_number = 1
    while True:
        time.sleep(random.randint(1, 8))
        current_url = base_url + str(page_number)+"&rand="+str(random.randint(1, 10000))+"&p=r"
        try:
            response = requests.get(url=current_url, headers=headers)
        except ConnectionError:
            print('连接失败，跳过该页:', current_url)
            continue
        tree = etree.HTML(response.text.encode('utf-8'))
        data_list = tree.xpath('/html/body/div[@class="c"]')
        if not data_list:
            break  # No more pages, exit the loop
        del data_list[-1]
        print('正在爬取第{}页'.format(page_number))
        for data in data_list:
            content, time_info, source, like_count, comment_count, repost_count = extract_info(data)
            csv_writer.writerow([content, time_info, source, like_count, comment_count, repost_count])
        page_number += 1

print('Extraction completed.')

In [ ]:
df = pd.read_csv('北京知识产权微博内容.csv')
# 创建新的列名列表
new_columns = ['微博内容', '时间', '来源', '点赞数', '评论数', '转发数']

# 将列名作为内容添加到新的行
df.loc[-1] = df.columns
df.index = df.index + 1
df.sort_index(inplace=True)

# 重命名列名
df.columns = new_columns
df.head(20)

In [ ]:
df.to_csv('北京知识产权微博内容.csv')

In [ ]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import requests
import pandas as pd
import random
from time import sleep

In [ ]:
#这个爬取没问题，但是会有连接时间问题，建议选时间跨度不要太大
province_text = '广东、河北、山西、辽宁、吉林、黑龙江、江苏、浙江、安徽、福建、江西、山东、河南、湖北、湖南、海南、四川、贵州、云南、陕西、甘肃、青海、台湾、内蒙古、广西、西藏、宁夏、新疆、北京、天津、上海、重庆、香港、澳门'
all_province_list = list(province_text.split('、'))
df = pd.DataFrame(dtype=str)

for province in all_province_list:
    for i in range(2020,2025):
        url = 'https://www.baiten.cn/results/filter'
        params = {
            'q': 'aa:((' + province + ')) AND (pd:[' + str(i) + '0101 TO ' + str(i) + '1231])',
            'fq': '',
            'sc': '35184372088831'
        }
        headers = {
            'Cookie': '_uab_collina=171636084297048745836638; yunsuo_session_verify=651468e84a027b9680fb898cb3261dac; JSESSIONID=467E8A8E43574AD6ADDD2CA1AA8E598B; zlcp2024423=true; BSESSION=e2148644ecde040ae13f44b2ace4184a38f0d6b0623961ad; PD=a6e2323d0e6adfd16b0bb9a0e28457c5d0c7bf66bf1f72327d2d56e14cb82af5a5f1e3e7ae91cf98c7fa13fa30f3d1fd'
        }
        resp = requests.post(url=url,params=params,headers=headers)
        page_json = resp.json()
        resp.close()
        
        cn_um,cn_id,cn_in,cn_gp,cn_in_gp = 0,0,0,0,0
        try:    
            cn_um = int(page_json['facetPivots']['country']['cn']['cn_um'])
        except:
            print(province,i,'实用新型专利')
        try:
            cn_id = int(page_json['facetPivots']['country']['cn']['cn_id'])
        except:
            print(province,i,'外观专利')
        try:
            cn_in = int(page_json['facetPivots']['country']['cn']['cn_in'])
        except:
            print(province,i,'发明专利')
        try:
            cn_gp = int(page_json['facetPivots']['country']['cn']['cn_gp'])
        except:
            print(province,i,'发明授权专利')
        try:
            cn_in_gp = int(page_json['facetPivots']['country']['cn']['cn_in_gp'])
        except:
            print(province,i,'发明公开专利')
        
        df = pd.concat([df,pd.DataFrame({'发明专利':cn_in,'发明公开专利':cn_in_gp,'发明授权专利':cn_gp,
                                         '外观专利':cn_id,'实用新型专利':cn_um,'年份':i,'省份':province},
                                       index=[len(df)])],axis=0)
    sleep(random.randint(1,3))
    
df
df.to_excel('./实验一爬取数据/佰腾网/佰腾网2020-2025全国专利数据.xlsx')
            


In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
start_date = "20250101"
end_date = "20250102"

# 创建一个空的DataFrame用于存储数据
df = pd.DataFrame(columns=['网址', '标题', '时间'])

# 设置请求头(headers)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51',
    'Cookie': 'Hm_lvt_162a2ae8b540393bd5792dda2692396e=1687362784; PHPSESSID=214cd4ed0774bc1bd19001d1640ebe75; uuid=CgI1QGSTHOEAMh3oECWbAg==; Hm_lpvt_162a2ae8b540393bd5792dda2692396e=1687447774'
}

# 循环遍历日期范围
current_date = start_date

while current_date <= end_date:
    date = pd.to_datetime(current_date, format="%Y%m%d").strftime("%Y-%m-%d")
    print('\n')
    print("===================" + date + "===================")
    url = f"https://bjrbdzb.bjd.com.cn/bjrb/mobile/2025/{current_date}/{current_date}_m.html#page0"
    response = requests.get(url, headers=headers, cookies=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 提取网址和标题
    links = soup.find_all('a', attrs={'data-href': True})
    for index, link in enumerate(links, start=1):
        mid_l = len(df.index)
        href = link['data-href']
        href = "https://bjrbdzb.bjd.com.cn/bjrb/mobile/2025/" + current_date + "/" + href[2:]  # 更新网址格式
        title = link.text.strip()

        # 将数据添加到DataFrame中
#         df = df.append({'网址': href, '标题': title, '时间': current_date}, ignore_index=True)
        df.loc[mid_l] = [href,title,current_date]


        response = requests.get(href, headers=headers, cookies=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        content_b = soup.find_all('b')
        b_texts = [tag.get_text() for tag in content_b]
        content_p = soup.find_all('p')
        p_texts = [tag.get_text() for tag in content_p]


        with open("./实验一爬取数据/北京日报/"+b_texts[0]+".txt",'w') as f:
            f.write(b_texts[0]+"\n")
            for i in p_texts:
                f.write(i+"\n")
        
        

        # 打印保存进度
        print(f"已将{date}第{index}篇文章中{title}写入北京日报文件夹")

    # 更新日期
    current_date = pd.to_datetime(current_date, format="%Y%m%d")
    current_date += pd.DateOffset(days=1)
    current_date = current_date.strftime("%Y%m%d")

# 将DataFrame保存到Excel文件
df.to_excel("./实验一爬取数据/Python实验一省机关报数据的抓取（以北京日报为例）.xlsx", index=False)

In [ ]:
print(links)